## Import All Necessary Modules and Adjust Settings

In [ ]:
# %matplotlib widget
%matplotlib inline
from datadriven import SqlQuery, list_fields, execute_query, list_tables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [18, 6]
pd.options.display.max_rows = 50

## List Fields from Results Table

In [ ]:
table_name = "fixed_fh_fn_results"
schema = "fixed"

In [ ]:
list_fields ("fixed_fh_fn_results")

,tablename,name,description
0,fixed_fh_fn_results,agency,Agency name
1,fixed_fh_fn_results,ca,"1 if CA, 0 is otherwise"
2,fixed_fh_fn_results,cash_win,"1 if cash window, 0 is otherwise"
3,fixed_fh_fn_results,cashout,"1 if Cashout, 0 otherwise"
4,fixed_fh_fn_results,cls,Current Loan Amount (Current UPB)
...,...,...,...
85,fixed_fh_fn_results,tpo,"1 if Third Party Origination (TPO), 0 is other..."
86,fixed_fh_fn_results,tx,"1 if TX, 0 is otherwise"
87,fixed_fh_fn_results,wala,Loan Age
88,fixed_fh_fn_results,wam,Remaining Months to Maturity (RMM)


In [ ]:
filters = "report_month >= '2019-08-01'"\
          " and product_pool in ('FNM30', 'FHLG30')"\
          " and coupon_pool between 2.5 and 12"\
          " and spec_pool_type in ('LLB', 'MLB', 'HLB', '200K', 'NY', 'TBA')"
hide_small_buckets = True
small_buckets_size = 3000
incentive_range = [-0.75, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1, 1.5]
wala_range = [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 18, 24, 36, 48, 90, 120, 180, 240, 360]
eps = 1e-10

## Calculating S-Surface

In [ ]:
query = SqlQuery("{schema}.{table} where {filters}".format(schema=schema, table=table_name, filters=filters))
query.create_field_range("incentive_rng", "incentive", incentive_range)
query.create_field_range("wala_rng", "wala", wala_range, strict=True)
query.select("incentive_rng is not null and wala_rng is not null")
query.create_tabulation(["incentive_rng", "wala_rng"], {"smm" : ("avg(cls)", "smm"), "loan_number" : ("sum", "loan_number")})
query.create_field("cpr", "smm_to_cpr(smm)")
if hide_small_buckets:
    query.select ("loan_number >= %s" % small_buckets_size)
df = execute_query(query)

In [ ]:
table = pd.pivot_table(df, values='cpr', columns=['incentive_rng'], index=['wala_rng'], aggfunc=np.mean)
table

incentive_rng,-0.75,-0.50,-0.40,-0.30,-0.20,-0.10,0.00,0.10,0.20,0.30,0.40,0.50,0.75,1.00,1.50
wala_rng,,,,,,,,,,,,,,,
1,NaN,NaN,0.607153,0.423151,0.540261,0.539041,0.613482,0.762564,0.697920,0.652846,0.576903,0.615881,0.535617,0.503921,0.920408
2,NaN,NaN,0.497282,0.927513,1.099512,0.648413,0.941132,0.992907,1.241760,1.360447,1.162434,1.308860,1.108067,0.830143,0.527241
3,NaN,NaN,1.211416,1.229579,2.180300,3.505578,3.518083,5.517834,5.605772,4.774267,6.157407,6.931156,6.450956,6.523324,7.008813
4,NaN,NaN,NaN,2.191271,1.710343,2.893096,4.895156,5.869610,7.103227,7.723057,8.528461,9.530433,9.015336,9.261448,9.106957
5,NaN,NaN,NaN,2.754635,2.857104,4.217463,3.630163,8.907807,7.446805,10.935862,12.132685,13.471264,13.831877,12.891203,12.807114
6,NaN,NaN,NaN,NaN,2.101204,5.625932,10.517172,12.630135,16.519669,20.386057,24.440266,28.118839,28.159067,25.782337,22.542385
7,NaN,NaN,NaN,NaN,NaN,17.361783,22.563078,19.584432,24.392749,31.267117,37.290079,45.506943,42.854107,40.216983,34.435753
8,NaN,NaN,NaN,NaN,NaN,NaN,28.085791,24.959116,29.937067,30.085529,38.386538,44.584459,44.676194,41.400231,38.560616
10,NaN,NaN,NaN,NaN,NaN,NaN,31.058688,32.288968,35.338698,36.380118,40.701620,44.586452,43.943181,42.827183,38.823108
